In [1]:
!hostname

midway3-0277.rcc.local


# Imports/Setpu

In [ ]:
# import sys
# import os
# import shutil

# username = os.environ.get('USER')
# scratch_base = f"/scratch/midway3/{username}/fea_project"
# custom_lib_path = os.path.join(scratch_base, "my_custom_libs")
# py_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
# site_packages = os.path.join(custom_lib_path, "lib", py_version, "site-packages")

# # Clean up old installation
# if os.path.exists(custom_lib_path):
#     print(f"Removing old libraries...")
#     shutil.rmtree(custom_lib_path)

# install_cmd = (
#     f"pip install --target='{site_packages}' "
#     "--upgrade "
#     "--no-cache-dir "
#     "'numpy<2.0' "
#     "'pandas' "
#     "'bottleneck' "
#     "'numexpr' "
#     "'torch' "
#     "'torchvision' "
#     "'accelerate>=0.31.0' "
#     "'tensorflow' "
#     "'tf-keras' "
#     "'transformers' "
#     "'sentence-transformers' "
#     "'datasets' "
#     "'botocore' "
#     "'aiobotocore' "
#     "'s3fs' "
#     "'openpyxl' "
#     "'sentencepiece' "
#     "'plotly' "
#     "'scikit-learn' "
#     "'papermill' "
#     "'scrapbook' "
#     "'openai' "
#     "'pydantic' "
#     "'tqdm' "
# )

# print("Installing with NumPy <2.0...")
# exit_code = os.system(install_cmd)

# if exit_code == 0:
#     print("\nSUCCESS: Complete stack installed.")
# else:
#     print(f"\nERROR: Installation failed with code {exit_code}")

# print("\n##### RESTART KERNEL NOW #####")

In [1]:
import sys
import os

### Installs all the packages in the install library (my_custom_libs) ##

# --- 1. SETUP PATHS FIRST (you are going to need to change to whatever your path to the instal library is in) ---
username = os.environ.get('USER')
scratch_base = f"/scratch/midway3/{username}/fea_project"
custom_lib_path = os.path.join(scratch_base, "my_custom_libs")
hf_cache_path = os.path.join(scratch_base, "hf_cache")

py_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
site_packages = os.path.join(custom_lib_path, "lib", py_version, "site-packages")
sys.path.insert(0, site_packages)

# FORCE custom path to the FRONT of the list
if site_packages not in sys.path:
    sys.path.insert(0, site_packages)

print(f"Python is looking here first: {sys.path[0]}")

# --- 2. ENV VARIABLES ---
os.environ['HF_HOME'] = hf_cache_path
os.environ['TRANSFORMERS_CACHE'] = hf_cache_path
os.environ['PYTHONUSERBASE'] = custom_lib_path

# --- 3. NOW IMPORT LIBRARIES ---
# Only import AFTER sys.path is updated
import datasets 
import sentence_transformers
from sentence_transformers import SentenceTransformer

# --- 4. VERIFY PATHS ---
print(f"Datasets is loaded from: {os.path.dirname(datasets.__file__)}")

if "software" in os.path.dirname(datasets.__file__):
    print("FAILURE: Still loading system datasets.")
else:
    print("SUCCESS: All libraries isolated in scratch.")

Python is looking here first: /scratch/midway3/aesteva/fea_project/my_custom_libs/lib/python3.11/site-packages
Datasets is loaded from: /scratch/midway3/aesteva/fea_project/my_custom_libs/lib/python3.11/site-packages/datasets
SUCCESS: All libraries isolated in scratch.


In [2]:
import papermill as pm
import scrapbook as sb
import pandas as pd
import os
from datetime import datetime
import importlib
import free_entailments_algorithm_utils as fea

# Global Variables:

In [3]:
#Change to False if running the real thing.

test = True

In [4]:

importlib.reload(fea)

#List of all sentences/sentence ids being used
df_p = pd.read_excel("ClauseLevel_df_p.xlsx")
df_sc = pd.read_excel("ClauseLevel_df_sc.xlsx")

In [5]:
## All clauses. Of the form: argument_id	sentence_id	sentence

df_clause = pd.concat([df_p,df_sc]).drop_duplicates(subset='sentence_id')

# We do the 0.5 billion pairs in batches:
# We compare books-books, books-speech, never speech-speech. 
# We also compare premise-premise, conclusion-conclusion, never premise-conclusion

if not test:
    df_pairs = fea.generate_valid_pairs(
            df_p,
            df_sc,
            'sentence_id',
            'sentence',
            max_pairs = 25,
        )

In [ ]:
# # Creates Embedding Cache (If not already created) from df_clause:
# import pickle

# cache_path = "embedding_cache_finetuned.pkl"

# if os.path.exists(cache_path):
#     with open(cache_path, 'rb') as f:
#         embedding_cache_finetuned = pickle.load(f)
#     print(f"Loaded existing embedding cache: {len(embedding_cache_finetuned)} embeddings")
# else:
#     # Create cache from all unique clauses
#     embedding_cache_finetuned = fea.create_embedding_cache(
#         df_texts=df_clause,
#         id_col='sentence_id',
#         text_col='sentence',
#         model_name="BAAI/bge-large-en-v1.5",
#         batch_size=128,
#         show_progress_bar=True
#     )
#     # Save to disk for future runs
#     with open(cache_path, 'wb') as f:
#         pickle.dump(embedding_cache_finetuned, f)
#     print(f"Saved embedding cache to {cache_path}")


In [6]:
input_file = "labeled_pairs/Results_DS_BtoS_iteration_0.csv"
df_input = pd.read_csv(input_file)

if test:
    importlib.reload(fea)
    df_input = pd.read_csv(input_file)
    
    input_file, remaining_llm_calls = fea.two_random_subsamples(
        df = df_input,
        frac1 = 0.5,
        frac2 = 0.5,
        random_state = 42
    )
else:
    remaining_llm_calls = None
    df_entailed_pairs = df_input.rename(columns={'sentence_id_1': 'id1', 'sentence_id_2': 'id2'})
    unlabeled_pairs = fea.setminus(df_pairs, df_entailed_pairs, ["id1", "id2"])

In [7]:
import pickle
with open("embedding_cache_finetuned.pkl", 'rb') as f:
    embedding_cache_finetuned = pickle.load(f)
print(f"Loaded embedding cache: {len(embedding_cache_finetuned)} embeddings")

Loaded embedding cache: 63909 embeddings


In [8]:
num_iterations = 3
total_cost = 0.0
iteration_stats = []

sent_frac = 0.5
budget = 0.0

# FEA Loop using Papermill

This notebook uses papermill to iteratively run the FEA_Pipeline notebook, feeding the output of each iteration as input to the next.

In [9]:
output_dir = "fea_iterations"
os.makedirs(output_dir, exist_ok=True)

start_iteration = 0

In [10]:
print(f"Starting FEA Loop with {num_iterations} iterations")
print("="*60)

Starting FEA Loop with 3 iterations


In [11]:
importlib.reload(fea)

iteration_stats = fea.run_fea_loop(
    test=test,
    input_file=input_file,
    df_clause=df_clause,
    embedding_cache=embedding_cache_finetuned,
    num_iterations=num_iterations,
    start_iteration=start_iteration,
    output_dir=output_dir,
    sent_frac=sent_frac,
    budget=budget,
    remaining_llm_calls=remaining_llm_calls,
    unlabeled_pairs=unlabeled_pairs if 'unlabeled_pairs' in dir() else None,
    deepseek_api_key=api_key,
)

VALIDATING INITIAL DATA STATE
Initial labeled pairs: 5000
Initial unlabeled pairs: 5000
Total: 10000
✓ Cleaned up old loop data directory
Starting with 5000 already labeled pairs
Unlabeled pool: 5000
Total pairs: 10000

ITERATION 0/3
Status: Labeled=5000, Unlabeled=5000, Total=10000
Executing FEA_Pipeline.ipynb with input: fea_iterations/loop_data/input_iter_0.csv


Executing:   0%|          | 0/26 [00:00<?, ?cell/s]

✓ Output has 103 NEW pairs
✓ Accumulated dataset now has 5103 total labeled pairs

VERDICT SUMMARY
Total pairs: 10000
Bidirectional entailment (YES): 1215 (12.2%)
Not bidirectionally entailed (NO): 8785 (87.8%)

    Bidirectionally entailed: 44/103 (42.7%)
  → Labeled input: 5000
  → Unlabeled available: 5000

ITERATION 1/3
Status: Labeled=5103, Unlabeled=4897, Total=10000
Executing FEA_Pipeline.ipynb with input: fea_iterations/loop_data/accumulated_labeled_iter_0.csv


Executing:   0%|          | 0/26 [00:00<?, ?cell/s]

✓ Output has 52 NEW pairs
✓ Accumulated dataset now has 5155 total labeled pairs

VERDICT SUMMARY
Total pairs: 10000
Bidirectional entailment (YES): 1215 (12.2%)
Not bidirectionally entailed (NO): 8785 (87.8%)

    Bidirectionally entailed: 13/52 (25.0%)
  → Labeled input: 5103
  → Unlabeled available: 4897

ITERATION 2/3
Status: Labeled=5155, Unlabeled=4845, Total=10000
Executing FEA_Pipeline.ipynb with input: fea_iterations/loop_data/accumulated_labeled_iter_1.csv


Executing:   0%|          | 0/26 [00:00<?, ?cell/s]

✓ Output has 27 NEW pairs
✓ Accumulated dataset now has 5182 total labeled pairs

VERDICT SUMMARY
Total pairs: 10000
Bidirectional entailment (YES): 1215 (12.2%)
Not bidirectionally entailed (NO): 8785 (87.8%)

    Bidirectionally entailed: 12/27 (44.4%)
  → Labeled input: 5155
  → Unlabeled available: 4845

ITERATION 3/3
Status: Labeled=5182, Unlabeled=4818, Total=10000
Executing FEA_Pipeline.ipynb with input: fea_iterations/loop_data/accumulated_labeled_iter_2.csv


Executing:   0%|          | 0/26 [00:00<?, ?cell/s]

✓ Output has 15 NEW pairs
✓ Accumulated dataset now has 5197 total labeled pairs

VERDICT SUMMARY
Total pairs: 10000
Bidirectional entailment (YES): 1215 (12.2%)
Not bidirectionally entailed (NO): 8785 (87.8%)

    Bidirectionally entailed: 2/15 (13.3%)
  → Labeled input: 5182
  → Unlabeled available: 4818

FEA Loop Complete!


In [12]:
stats_df = fea.save_iteration_stats(iteration_stats, output_dir)


Iteration Statistics:
   iteration  labeled_input  unlabeled_available  pairs_selected  \
0          0           5000                 5000             103   
1          1           5103                 4897              52   
2          2           5155                 4845              27   
3          3           5182                 4818              15   

   cumulative_labeled  unlabeled_remaining  \
0                5103                 4897   
1                5155                 4845   
2                5182                 4818   
3                5197                 4803   

                           output_file  entailment_ratio  
0  fea_iterations/llm_batch_iter_0.csv          0.427184  
1  fea_iterations/llm_batch_iter_1.csv          0.250000  
2  fea_iterations/llm_batch_iter_2.csv          0.444444  
3  fea_iterations/llm_batch_iter_3.csv          0.133333  

COST SUMMARY
Total pairs sent to LLM: 197
Cost per pair: $0.001900
Total cost: $0.3743

✓ Saved iteration sta